In [1]:
import os
import csv

In [2]:
#! svn checkout svn://svn.code.sf.net/p/diseaseontology/code/trunk/ download

In [3]:
import sys
sys.path.append('IGS_scripts')

import oboparser

In [4]:
#import imp
#imp.reload(oboparser)
path = os.path.join('download', 'HumanDO.obo')
do = oboparser.parse(path, ['is_a'])

In [5]:
do.get_term_ids()
do.get_term('DOID:0050234')

In [6]:
term = do.get_term('DOID:2377')
term.__dict__

{'alternateIds': [],
 'definition': '"A demyelinating disease that involves damage to the fatty myelin sheaths around the axons of the brain and spinal cord resulting in demyelination and scarring." [url:http\\://en.wikipedia.org/wiki/Multiple_sclerosis]',
 'id': 'DOID:2377',
 'name': 'multiple sclerosis',
 'obsolete': False,
 'relationships': [('is_a', 'DOID:3213', 'demyelinating disease')],
 'subsets': [],
 'synonyms': [('Generalized multiple sclerosis (disorder)',
   'EXACT',
   'SNOMEDCT_2005_07_31:192928003'),
  ('insular sclerosis', 'EXACT', 'CSP2005:2042-2324')],
 'xrefs': ['EFO:0003885',
  'ICD9CM:340',
  'MSH:D009103',
  'NCI:C3243',
  'OMIM:126200',
  'OMIM:612594',
  'OMIM:612595',
  'OMIM:612596',
  'OMIM:614810',
  'SNOMEDCT_2010_1_31:155023009',
  'SNOMEDCT_2010_1_31:192928003',
  'SNOMEDCT_2010_1_31:192930001',
  'SNOMEDCT_2010_1_31:24700007',
  'UMLS_CUI:C0026769']}

In [7]:
# Create a networkx representation
import networkx
terms = do.get_terms()
dox = networkx.MultiDiGraph()
dox.add_nodes_from(term for term in terms if not term.obsolete)
for term in dox:
    for typedef, id_, name in term.relationships:
        dox.add_edge(term, do.get_term(id_), key = typedef)

assert networkx.is_directed_acyclic_graph(dox)

In [8]:
term.__dict__

{'alternateIds': [],
 'definition': '',
 'id': 'DOID:4756',
 'name': 'testicular Leydig cell tumor',
 'obsolete': False,
 'relationships': [('is_a', 'DOID:2696', 'Leydig cell tumor')],
 'subsets': [],
 'synonyms': [('testicular Leydig cell neoplasm',
   'EXACT',
   'NCI2004_11_17:C6356')],
 'xrefs': ['NCI:C6356', 'UMLS_CUI:C0863027']}

In [57]:
def write_xref_row(writer, doid_code, doid_name, xrefs):
    rows = list()
    for xref in xrefs:
        resource, resource_id = xref.split(':', 1)
        rows.append([doid_code, doid_name, resource, resource_id])
    rows.sort()
    writer.writerows(rows)

file_unprop = open(os.path.join('data', 'xrefs.tsv'), 'w')
file_prop = open(os.path.join('data', 'xrefs-prop.tsv'), 'w')

writer_unprop = csv.writer(file_unprop, delimiter='\t')
writer_prop = csv.writer(file_prop, delimiter='\t')

for writer in writer_unprop, writer_prop:
    writer.writerow(['doid_code', 'doid_name', 'resource', 'resource_id'])

for term in networkx.topological_sort_recursive(dox, reverse=True):
    xrefs = set(term.xrefs)
    xrefs_prop = set(xrefs)
    for ancestor in networkx.ancestors(dox, term):
        xrefs_prop |= set(ancestor.xrefs)
    
    write_xref_row(writer_unprop, term.id, term.name, xrefs)
    write_xref_row(writer_prop, term.id, term.name, xrefs_prop)

for write_file in file_unprop, file_prop:
    write_file.close()


In [13]:
rows = list()
for term in dox:
    rows.append({'doid': term.id, 'name': term.name, 'type': 'name'})
    for synonym in term.synonyms:
        rows.append({'doid': term.id, 'name': synonym[0], 'type': '{}-synonym'.format(synonym[1].lower())})
path = os.path.join('data', 'term-names.tsv')
with open(path, 'w') as write_file:
    writer = csv.DictWriter(write_file, delimiter='\t', fieldnames=['doid', 'name', 'type'])
    writer.writeheader()
    writer.writerows(rows)